In [1]:
!pip install transformers datasets torch tqdm pandas kss scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 9.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.7/939.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 9.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.8 MB/s eta 0:00:00
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16256 sha256=cd33b319d46cddac5453c0988d90f9aa66d5337ecf63083f4c7a82488de695e2
  Stored in directory: /Users/eunjung/Library/Caches/pip/wheels/24/a8/58/407063d8e5c1d4dd6594c99d12baa0108570b56a92325587dd
  Created wheel for pecab: filename=pecab-1.0.8-py3-none-any.whl size=26646665 sha256=d256b517a5e59a4e6af87bd1dd6c7c7a4f31bc426c232b3edbd0b7801be98baa
  Stored in directory: /Users/eunjung/L

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

import pandas as pd
df = pd.read_csv("crawling/naver_blog_reviews_filtered.csv")

MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)


# 감성 분석 함수 정의
def analyze_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = F.softmax(outputs.logits, dim=1)
    pred = torch.argmax(probs, dim=1).item()

    if pred == 0:
        return "부정"
    elif pred == 1:
        return "긍정"
    else:
        return "중립"
    
# 데이터에 감성 태그 붙이기
from tqdm import tqdm
sentiments = []

for title in tqdm(df['제목'].dropna()):
    sentiment = analyze_sentiment(title)
    sentiments.append(sentiment)

df['감성'] = sentiments
print(df[['제목', '감성']].head())

# 감성 분포 시각화
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = 'Malgun Gothic'

plt.figure(figsize=(6, 4))
sns.countplot(x='감성', data=df, palette='Set2')
plt.title("감성 분류 결과")
plt.show()

# 감성 분석 결과 저장
df.to_csv("인공지능_블로그_감성분석결과.csv", index=False, encoding='utf-8-sig')

tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyError: '제목'

model.safetensors:   0%|          | 0.00/452M [00:00<?, ?B/s]

In [ ]:
# # 모델 로드
# model = AutoModelForSequenceClassification.from_pretrained(
#     model_name,
#     num_labels=3  # 감정 카테고리 수
# )

In [ ]:
# # 학습 설정 & Trainer 객체 생성
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     learning_rate=5e-5,
#     per_device_train_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     logging_dir="./logs",
#     logging_steps=10
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=dataset,
#     eval_dataset=dataset
# )

# trainer.train()

In [ ]:
# text = "이 식당 진짜 맛있어요"
# inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
# with torch.no_grad():
#     outputs = model(**inputs)
#     logits = outputs.logits
#     pred = torch.argmax(logits, dim=-1).item()

# print("감정 라벨:", pred)  # 0=부정, 1=중립, 2=긍정